In [1]:
import ast

from riotwatcher import LolWatcher
import pandas as pd
import time
from tqdm import tqdm

from scripts.download_matches import lol_watcher

In [10]:
import pandas as pd
from collections import Counter

# Load data
df = pd.read_parquet('../data/match_drafts.parquet')



print(f"Total matches: {len(df)}")
print(f"Blue team wins: {df['blue_win'].sum()} ({df['blue_win'].mean()*100:.1f}%)")
print(f"Red team wins: {len(df) - df['blue_win'].sum()} ({100 - df['blue_win'].mean()*100:.1f}%)")

print("\n" + "=" * 50)
print("MOST PICKED CHAMPIONS")
print("=" * 50)

all_picks = []
for pick in df['red_picks']:
    all_picks.extend(pick.tolist())

for pick in df['blue_picks']:
    all_picks.extend(pick.tolist())

# Count and show top 10
pick_counts = Counter(all_picks)
for champ, count in pick_counts.most_common(10):
    print(f"{champ:20} {count:3} picks")

Total matches: 100
Blue team wins: 55 (55.0%)
Red team wins: 45 (45.0%)

MOST PICKED CHAMPIONS
Jayce                 23 picks
Riven                 22 picks
Ambessa               21 picks
Nami                  18 picks
Nautilus              18 picks
Kaisa                 17 picks
Jax                   16 picks
Khazix                16 picks
Corki                 16 picks
Ryze                  16 picks


In [12]:
print("\n" + "=" * 50)
print("CHAMPION WIN RATES (Min 5 games)")
print("=" * 50)

# Calculate win rates
champ_wins = {}
champ_games = {}

for _, row in df.iterrows():
    blue_champs = row['blue_picks']
    red_champs = row['red_picks']
    blue_won = row['blue_win']

    # Count wins for blue team
    for champ in blue_champs:
        champ_games[champ] = champ_games.get(champ, 0) + 1
        if blue_won:
            champ_wins[champ] = champ_wins.get(champ, 0) + 1

    # Count wins for red team
    for champ in red_champs:
        champ_games[champ] = champ_games.get(champ, 0) + 1
        if not blue_won:
            champ_wins[champ] = champ_wins.get(champ, 0) + 1

# Calculate win rates
champ_winrates = {}
for champ in champ_games:
    if champ_games[champ] >= 5:  # At least 5 games
        winrate = champ_wins.get(champ, 0) / champ_games[champ] * 100
        champ_winrates[champ] = winrate

# Show top and bottom 5
sorted_winrates = sorted(champ_winrates.items(), key=lambda x: x[1], reverse=True)

print("\nHighest win rates:")
for champ, wr in sorted_winrates[:5]:
    games = champ_games[champ]
    print(f"{champ:20} {wr:.1f}% ({games} games)")

print("\nLowest win rates:")
for champ, wr in sorted_winrates[-5:]:
    games = champ_games[champ]
    print(f"{champ:20} {wr:.1f}% ({games} games)")



CHAMPION WIN RATES (Min 5 games)

Highest win rates:
Alistar              92.9% (14 games)
Sivir                80.0% (5 games)
Ashe                 80.0% (5 games)
Riven                77.3% (22 games)
TwistedFate          75.0% (12 games)

Lowest win rates:
Varus                16.7% (12 games)
Poppy                16.7% (6 games)
Leona                12.5% (8 games)
Lux                  11.1% (9 games)
Malphite             0.0% (5 games)

✅ Data looks good! Ready for modeling.
